## Truthiness

### Expressions and Conditionals

Short circuiting the `&&` and `||` operators exhibit a behavior called `short circuiting`, which means it will stop evaluating expressions once it can guarantee the return value.<br><br>

The `&&` will short circuit when it encounters the first `false` expressions

```ruby
false && 3/0
=> false
```

Notice the above code doesn't generate a `ZeroDivisionError`. This is because the `&&` operator didn't even evaluate the second expression; since the first expression is `false`, it can short circuit and return `false`.

Also, notice that `false || 3/0` will generate an error.

```ruby
false || 3/0
# ZeroDivisionError: divided by 0
```

The `||` will short circuit when it encounters the first true expression.

### Truthiness

**Ruby is a very liberal language and considers everything to be truthy other than `false` and `nil`.**

It does **not**, however, mean that  `5 == true`:

```ruby
num = 5
num == true        # => false
```

## Flowchart & Pseudocode

Asking the user to give us N collections of numbers. We want to take the largest number out of each collection, and display it.

high-level psuedo code:

```
while user wants to keep going
  - ask the user for a collection of numbers
  - extract the largest one from that collection and save it  -- Subprocess
  - ask the user if they want to input another collection

return saved list of numbers
```

Translating shortened pseudo-code into formal pseudo-code:

```
START

SET large_numbers = []
SET keep_going = true

WHILE keep_going == true
  GET "enter a collection"
  SET collection
  SET largest_number = SUBPROCESS "extract the largest one from that collection"
  large_numbers.push(largest_number)
  GET "enter another collection?"
  IF "yes"
    keep_going = true
  ELSE
    keep_going = false
  IF keep_going == false
    exit the loop

PRINT large_numbers

END
```

A flowchart for this peusdo code would look like this:

<img src="flowchart_example.jpeg" width=500>

## Precedence

### Evaluation Order

The ternary operator (`?:`) and the short-circuit operators `&&` and `||` are a common source of unexpected behavior where precedence is concerned. Consider the following expressions:

```ruby
3 ? 1 / 0 : 1 + 2  # raises error ZeroDivisionError
5 && 1 / 0         # raises error ZeroDivisionError
nil || 1 / 0       # raises error ZeroDivisionError
```

What happens, though, if we modify things so that `1 / 0` isn't needed?

```ruby
nil ? 1 / 0 : 1 + 2  # 3
nil && 1 / 0         # nil
5 || 1 / 0           # 5
```

In all 3 cases, `1 / 0 `never gets executed, even though operator precedence would suggest that it should be evaluated first. 
- In the first expression, `1 / 0` isn't evaluated since it's the truthy operand for the `?:` - it only gets run when the value to the left of `?` is truthy. Instead, the code returns `3 (1 + 2)`.
- The other two expressions don't evaluate `1 / 0` due to `short-circuiting`. In all 3 expressions, this is simply the way Ruby works - it treats `?:`, `&&`, and `||` differently from other operators and doesn't evaluate subexpressions unless it needs them.

### Diving Deeper

In [4]:
array = [1, 2, 3]
p array.map { |num| num + 1 } # The return value of `map` then gets passed into p as an argument, which outputs `[2, 3, 4]`.

[2, 3, 4]


[2, 3, 4]

In [5]:
array = [1, 2, 3]
p array.map do |num|
  num + 1
end

#<Enumerator: [1, 2, 3]:map>


#<Enumerator: [1, 2, 3]:map>

Blocks have the lowest precedence of all operators. But between the two, `{ }` has slightly higher precedence than `do...end`. This has an effect on which method call the block gets passed to.

With `do...end` being the “weakest” of all the operators, `array.map` gets bound to `p`, which first invokes `array.map`, returning an `Enumerator` object. The `Enumerator` is then passed to `p`, along with the block. `p` prints the `Enumerator`, but doesn't do anything with the block.

In other words, the binding between a method name and a method's argument (`p` and the return value of `array.map`) is slightly tighter than the binding between a method call and a `do...end` block. Thus, `array.map` gets executed first, then the return value and the block get passed to `p` as separate arguments.

A `{}` block, on the other hand, has **higher priority** which means that it binds more tightly to `array.map`. Therefore, when we use `{},` `array.map `is called with the block, then the return value of `array.map` gets passed to `p`.

A visualization of both scenarios:

```ruby
array = [1, 2, 3]

p(array.map) do |num|
  num + 1                           #  <Enumerator: [1, 2, 3]:map>
end                                 #  => <Enumerator: [1, 2, 3]:map>

p(array.map { |num| num + 1 })      # [2, 3, 4]
                                    # => [2, 3, 4]
```

## Variable Scope

### Variables and Blocks

```ruby
[1, 2, 3].each do |num|
  puts num
end
```

The part of the code that we call the *block* is the part following the `#each` method invocation:

The `do..end` can be replaced by `{..}`.

**Blocks create a new scope for local variables. You can think of the scope created by a block following a method invocation as an *inner scope***

Nested blocks will create nested scopes. A variable's scope is determined by where it is initialized.

**Variables initialized in an outer scope can be accessed in an inner scope, but not vice versa.**

In [7]:
# Example 1: Outer scope can be accessed by inner scope

a = 1

loop do 
  puts a 
  a += 1
  break
end

puts a

1
2


In [8]:
# Example 2: inner scope variables cannot be accessed in outer scope

loop do   # the block following the invocation of the `loop` method creates an inner scope
  b = 1
  break
end

puts b

NameError: undefined local variable or method `b' for #<Object:0x00007fc715b322f0>

In [ ]:
# Example 3: peer scopes do not conflict

2.times do
  a = 'hi'
  puts a 
end

loop do 
  puts a 
  break
end

# Peer blocks cannot reference variables initialized in other blocks. 
# This means that we could use the same variable name `a` in the block of code that follows 
# the `loop` method invocation. However, it's not the same variable as in the first block.

In [ ]:
# Example 4: nested blocks

a = 1           # first level variable

loop do         # second level
  b = 2

  loop do       # third level
    c = 3
    puts a      # => 1
    puts b      # => 2
    puts c      # => 3
    break
  end

  puts a        # => 1
  puts b        # => 2
  puts c        # => NameError
  break
end

puts a          # => 1
puts b          # => NameError
puts c          # => NameError

In [13]:
# Example 5: variable shadowing

n = 10  
 
[1, 2, 3].each do |n|
  puts n  # The puts n will use the block parameter n and disregard the outer scoped local variable.
end

1
2
3


[1, 2, 3]

In [14]:
n = 10

1.times do |n| # Variable Shadowing
  n  = 11 
end

puts n          # => 10

10


### Variables and Method Definitions

While a block can access variables that were initialized outside of the block, **a method cannot -- its scope is self-contained.**

Methods can only access variables that were initialized inside the method or that are defined as parameters.

**local variables that are not initialized inside a method definition must be defined as parameters.**

In [15]:
# Example 1: a method definition can't access local variables in another scope

a = 'hi'

def some_method
  puts a
end

# invoke the method
some_method     # => NameError: undefined local variable or method `a' for main:Object

NameError: undefined local variable or method `a' for #<Object:0x00007fc715b322f0>

In [16]:
# Example 2: a method definition can access objects passed in

def some_method(a)
  puts a
end

some_method(5)  # => 5

5


In [17]:
# Ruby will first search for the local variable, and if it is not found, then Ruby tries to find a method with the given name. 
# If neither local variable nor method is found, then a `NameError` message will be thrown. 

hello = 'hi'

def hello
  "Saying hello!"
end

puts hello

hi


### Blocks within Method Definitions

The rules of scope for a method invocation with a block remain in full effect even if we're working inside a method definition.

In [ ]:
def some_method
  a = 1
  5.times do
    puts a
    b = 2 # Variables initialized within the inner scope of a block cannot be accessed by the outer scope.
  end

  puts a
  puts b
end

some_method     # => NameError: undefined local variable or method `b' for main:Object

### Constants

The scoping rules for constants are not the same as local variables. In procedural style programming, constants behave like globals.

Constants are said to have **lexical scope**

In [19]:
USERNAME = 'Batman'

def authenticate
  puts "Logging in #{USERNAME}"
end

authenticate    # => Logging in Batman

Logging in Batman


## More Variable Scope

A block is part of the method invocation. In fact, method invocation followed by curly braces or `do..end` is the way in which we define a block in Ruby.

Essentially the block acts as an argument to the method. In the same way that a local variable can be passed as an argument to a method at invocation, when a method is called with a block, the block acts as an argument to that method.

In [20]:
# Example 1: method parameter not used

def greetings(str)
  puts "Goodbye"
end

word = "Hello"

greetings(word)

Goodbye


In [21]:
# Example 2: method parameter used

def greetings(str)
  puts str
  puts "Goodbye"
end

word = "Hello"

greetings(word)

Hello
Goodbye


In [22]:
# Example 3: block not executed

def greetings
  puts "Goodbye"
end

word = "Hello"

greetings do
  puts word
end

Goodbye


In [23]:
# Example 4: block executed

def greetings
  yield
  puts "Goodbye"
end

word = "Hello"

greetings do
  puts word
end

Hello
Goodbye


In Example 3 the `greetings` method is invoked with a block, but the method is not defined to use a block in any way and so the block is not executed.

In Example 4 the `yield` keyword is what controls the interaction with the block, in this case it executes the block once. Since the block has access to the local variable `word`, `Hello` is output when the block is executed. 

Blocks and methods can interact with each other; the level of that interaction is set by the method definition and then used at method invocation.

When invoking a method with a block, we aren't just limited to executing code within the block; depending on the method definition, the method can use the *return value* of the block to perform some other action.

In [24]:
a = "hello"

[1, 2, 3].map { |num| a }

["hello", "hello", "hello"]

The `Array#map` method is defined in such a way that it uses the return value of the block to perform transformation on each element in an array. In the above example, the `#map` method doesn't have direct access to the `a` variable. However, the block that we pass to map (and that map calls) does have access to `a`. Thus, the block can use the value of `a` to determine the transformation value for each array element.

How a method definition accesses local variables compared to how a method invocation with a block accesses local variables:

Method definitions cannot directly access local variables initialized outside of the method definition, nor can local variables initialized outside of the method definition be reassigned from within it.

A block **can** access local variables initialized outside of the block and can reassign those variables. We already know that methods can access local variables passed in as arguments, and now we have seen that methods can access local variables through interaction with blocks.

We can think of **method definition** as setting a certain scope for any local variables in terms of the parameters that the method definition has, what it does with those parameters, and also how it interacts (if at all) with a block. 

We can then think of **method invocation** as using the scope set by the method definition. If the method is defined to use a block, then the scope of the block can provide additional flexibility in terms of how the method invocation interacts with its surroundings.

## Pass by Reference vs Pass by Value

### What Ruby does

Ruby exhibits a combination of behaviors from both "pass by reference" as well as "pass by value". Some people call this **pass by value of the reference** or **call by sharing**. 

**when an operation within the method mutates the caller, it will affect the original object**

### [Variables as Pointers](https://launchschool.com/books/ruby/read/more_stuff#variables_as_pointers)

The variable doesn't actually contain the value. Instead, it contains a pointer to a specific area in memory that contains the value.

In [27]:
a = "hi there"
b = a
a = "not here"  # Reassigning a to a different address in memory

puts b

hi there


In [28]:
a = "hi there"
b = a
a << ", Bob"  # Mutating the caller (a).  Since a is still pointing to the same address in memory as b 
              # this will also mutate b

puts b

hi there, Bob


**variables are pointers to physical space in memory**

**some operations mutate the address space, while others simply make the variable point to a different address space.**

In [176]:
s.split.each { |word| word.reverse if word.size >= 5 }

["Walk", "around", "the", "block"]

In [211]:
def stringy_2(size)
  string = ''
  size.times { |idx| idx.even? ? stringy << '1' : stringy << '0' }
end

SyntaxError: (irb):1: syntax error, unexpected end-of-input, expecting `end'

In [210]:
stringy_2(9)

9